In [1]:
import pandas as pd
import numpy as np 
file1 = r"databricks.csv"
# Read CSV files into DataFrames

# Read CSV files into DataFrames
df_full = pd.read_csv(file1)


In [8]:
df_full

,variant_id,store_id,Order_Number,quantity_delivery,quantity_sales,unit_price_delivery,STATUS,unit_price_sales,checkout_completed_at,state,LIB_ETAT_ARTICLE,GRP,DEP,RAY,SAISONNALITE_ARTICLE,stockout,mois
0,398517,25,202654,1.0,2.0,916.0,found,374.0,2025-06-03T00:34:35.000Z,fulfilled,Actif,FOOD,PRODUIT FRAIS,FRUITS ET LEGUMES,NaN,0,06
1,397028,25,205707,NaN,3.0,NaN,unfound,1720.0,2025-06-13T14:20:13.000Z,fulfilled,Actif,FOOD,PGC,BISCUITERIE,PERMANENT,1,06
2,392891,25,206424,NaN,1.0,NaN,unfound,2085.0,2025-06-15T12:20:43.000Z,cancelled,Actif,FOOD,PGC,BISCUITERIE,PERMANENT,1,06
3,398786,25,211692,1.0,1.0,3780.0,found,3780.0,2025-06-29T21:23:28.000Z,fulfilled,Actif,FOOD,PRODUIT FRAIS,CREMERIE,PERMANENT,0,06
4,1904234,25,212068,1.0,1.0,1801.0,found,1298.0,2025-06-30T17:30:55.000Z,fulfilled,Actif,FOOD,PRODUIT FRAIS,FRUITS ET LEGUMES,NaN,0,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79026,444452,25,212862,1.0,NaN,3395.0,new/replaced,NaN,2025-07-03T00:07:53.000Z,fulfilled,Gele achat,FOOD,PRODUIT FRAIS,CREMERIE,PERMANENT,0,07
79027,398307,25,215750,1.0,NaN,5095.0,new/replaced,NaN,2025-07-11T11:45:23.000Z,fulfilled,Actif,FOOD,PRODUIT FRAIS,CREMERIE,PERMANENT,0,07
79028,395137,25,213931,1.0,NaN,4295.0,new/replaced,NaN,2025-07-05T21:33:32.000Z,fulfilled,Actif,FOOD,PRODUIT FRAIS,CREMERIE,PERMANENT,0,07
79029,395219,25,214672,1.0,NaN,2995.0,new/replaced,NaN,2025-07-08T00:00:18.000Z,fulfilled,Actif,FOOD,PGC,BEAUTE-SANTE,PERMANENT,0,07


In [2]:
# 1) Nom de la colonne contenant la date
DATE_COL = "checkout_completed_at"   # adapte si ta colonne s’appelle autrement, p. ex. "date"

# 2) Conversion explicite en datetime (sécurise le type)
df_full[DATE_COL] = pd.to_datetime(df_full[DATE_COL], errors="coerce")  # errors="coerce" mettra NaT si format invalide

# 3) Extraction des bornes
date_min = df_full[DATE_COL].min()
date_max = df_full[DATE_COL].max()

print("📅 Date minimale :", date_min)
print("📅 Date maximale :", date_max)


📅 Date minimale : 2024-01-01 00:00:00
📅 Date maximale : 2025-07-20 00:00:00


In [42]:
df_full.columns

Index(['variant_id', 'store_id', 'Order_Number', 'quantity_delivery',
       'quantity_sales', 'unit_price_delivery', 'STATUS', 'unit_price_sales',
       'checkout_completed_at', 'state', 'LIB_ETAT_ARTICLE', 'GRP', 'DEP',
       'RAY', 'SAISONNALITE_ARTICLE', 'stockout', 'mois'],
      dtype='object')

### Check the data types of the numeric columns.

In [27]:
import pandas as pd

# ------------------------------------------------------------------------------
# 0. Exemple : votre DataFrame s’appelle df_full
# ------------------------------------------------------------------------------

# 1) Colonnes que vous considérez comme numériques
num_cols = [
    "variant_id", "store_id", "Order_Number",
    "quantity_delivery", "quantity_sales",
    "stockout"         # 0/1 ou NaN
]

# 2) Détection des valeurs non numériques
rapport = {}  # stockera pour chaque colonne la liste des valeurs problématiques

for col in num_cols:
    # Tentative de conversion → NaN pour tout ce qui n’est pas numérique
    coerced = pd.to_numeric(df_full[col], errors="coerce")
    
    # Index des lignes où la coercition a échoué (non numérique ET non vide)
    mask_bad = coerced.isna() & df_full[col].notna()
    
    if mask_bad.any():
        # On enregistre les valeurs fautives (unique pour la lisibilité)
        mauvaises_valeurs = df_full.loc[mask_bad, col].unique().tolist()
        rapport[col] = mauvaises_valeurs

# 3) Rapport final
if not rapport:
    print("✅ Toutes les colonnes numériques ne contiennent que des nombres.")
else:
    print("⚠️  Valeurs non numériques détectées :")
    for col, vals in rapport.items():
        print(f" - {col} : {len(vals)} valeur(s) non numérique(s) → {vals[:10]}{' ...' if len(vals)>10 else ''}")


✅ Toutes les colonnes numériques ne contiennent que des nombres.


#### 🕒 Date Cleaning: Convert and Normalize 'checkout_completed_at' to Date Only

In [28]:
import pandas as pd

# 1) Passe l’horodatage à la date civile (hh:mm:ss → 00:00:00)
df_full["checkout_completed_at"] = pd.to_datetime(df_full["checkout_completed_at"], errors="coerce").dt.normalize()




In [62]:
df_full

,variant_id,store_id,Order_Number,quantity_delivery,quantity_sales,unit_price_delivery,STATUS,unit_price_sales,checkout_completed_at,state,LIB_ETAT_ARTICLE,GRP,DEP,RAY,SAISONNALITE_ARTICLE,stockout,mois,date
0,423367,25,193482,1.0,1.0,730.0,found,695.0,2025-05-06 00:00:00+00:00,fulfilled,Actif,FOOD,PRODUIT FRAIS,FROMAGERIE,PERMANENT,0,05,2025-05-06 00:00:00+00:00
1,393695,25,201722,1.0,1.0,1990.0,found,1990.0,2025-05-31 00:00:00+00:00,fulfilled,Actif,FOOD,PRODUIT FRAIS,CREMERIE,PERMANENT,0,05,2025-05-31 00:00:00+00:00
2,397494,25,199654,1.0,1.0,2495.0,found,2495.0,2025-05-25 00:00:00+00:00,fulfilled,Actif,FOOD,PGC,EPICERIE,PERMANENT,0,05,2025-05-25 00:00:00+00:00
3,398515,25,197197,1.0,4.0,712.0,found,174.0,2025-05-18 00:00:00+00:00,fulfilled,Actif,FOOD,PRODUIT FRAIS,FRUITS ET LEGUMES,NaN,0,05,2025-05-18 00:00:00+00:00
4,397533,25,199337,50.0,50.0,180.0,found,180.0,2025-05-25 00:00:00+00:00,fulfilled,Actif,FOOD,PGC,LIQUIDE,PERMANENT,0,05,2025-05-25 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142481,444452,25,212862,1.0,NaN,3395.0,new/replaced,NaN,2025-07-03 00:00:00+00:00,fulfilled,Gele achat,FOOD,PRODUIT FRAIS,CREMERIE,PERMANENT,0,07,2025-07-03 00:00:00+00:00
142482,398307,25,215750,1.0,NaN,5095.0,new/replaced,NaN,2025-07-11 00:00:00+00:00,fulfilled,Actif,FOOD,PRODUIT FRAIS,CREMERIE,PERMANENT,0,07,2025-07-11 00:00:00+00:00
142483,395137,25,213931,1.0,NaN,4295.0,new/replaced,NaN,2025-07-05 00:00:00+00:00,fulfilled,Actif,FOOD,PRODUIT FRAIS,CREMERIE,PERMANENT,0,07,2025-07-05 00:00:00+00:00
142484,395219,25,214672,1.0,NaN,2995.0,new/replaced,NaN,2025-07-08 00:00:00+00:00,fulfilled,Actif,FOOD,PGC,BEAUTE-SANTE,PERMANENT,0,07,2025-07-08 00:00:00+00:00


## Feature Engineering: Creating predictive variables from raw data

### Daily aggregation (total_demand, stockout_occurred)

In [29]:


# ------------------------------------------------------------------
# 1. Agrégation quotidienne (total_demand, stockout_occurred)
# ------------------------------------------------------------------
daily_data = (df_full
              .groupby(["variant_id", "checkout_completed_at"], as_index=False)
              .agg(total_demand       = ("quantity_sales", "sum"),
                   stockout_occurred  = ("stockout",        "max"))
             )


### Preparation: chronological order within each product

In [30]:

# ------------------------------------------------------------------
# 2. Préparation : ordre chronologique dans chaque produit
# ------------------------------------------------------------------
daily_data.sort_values(["variant_id", "checkout_completed_at"], inplace=True)

# Numéro de ligne équivalent à row_number() over (partition bpp variant_id, order by date)
daily_data["row_num"] = daily_data.groupby("variant_id").cumcount() + 1




In [31]:
# Assure-toi que daily_data est trié par produit + date
daily_data = daily_data.sort_values(["variant_id", "checkout_completed_at"])

# Moyenne glissante des 7 jours précédents (t‑1 … t‑7)
daily_data["demand_7day_avg"] = (
    daily_data
      .groupby("variant_id")["total_demand"]
      .transform(lambda s: s.shift(1).rolling(7, min_periods=1).mean())
)


In [32]:
# Stock‑out dans les 7 jours suivants (t+1 … t+7)
daily_data["stockout_risk_target"] = (
    daily_data
      .groupby("variant_id")["stockout_occurred"]
      .transform(lambda s: s.shift(-1).rolling(7, min_periods=1).max())
)


In [33]:
import pandas as pd

# -------------------------------------------------------------
# 1. Définis la liste des colonnes à rattacher
#    (complète‑la si tu en as d’autres)
# -------------------------------------------------------------
cat_cols = [
    "DEP",
    "GRP",
    "RAY",
    "store_id",
    "SAISONNALITE_ARTICLE",
    "state",
    "LIB_ETAT_ARTICLE",
    "STATUS"
]

# -------------------------------------------------------------
# 2. Construis le "lookup" de référence
#    → 1 ligne unique par (variant_id, checkout_completed_at)
# -------------------------------------------------------------
cat_lookup = (
    df_full[["variant_id", "checkout_completed_at", *cat_cols]]
      .drop_duplicates(subset=["variant_id", "checkout_completed_at"])
      # .groupby(["variant_id", "checkout_completed_at"], as_index=False)[cat_cols].first()
      # ↳ Variante si tu préfères forcer "first()" et être sûr d'avoir une seule ligne
)

# Harmonise le type datetime si besoin
for d in (daily_data, cat_lookup):
    d["checkout_completed_at"] = (
        pd.to_datetime(d["checkout_completed_at"], errors="coerce").dt.tz_localize(None)
    )

# -------------------------------------------------------------
# 3. Jointure LEFT (équivalent du .join(..., how="left") PySpark)
# -------------------------------------------------------------
daily_data = daily_data.merge(
    cat_lookup,
    on=["variant_id", "checkout_completed_at"],
    how="left",
    validate="many_to_one"   # daily_data → many ; cat_lookup → ≤1
)

print("✅ Colonnes ajoutées :", [c for c in cat_cols if c in daily_data.columns])
print("Shape :", daily_data.shape)


✅ Colonnes ajoutées : ['DEP', 'GRP', 'RAY', 'store_id', 'SAISONNALITE_ARTICLE', 'state', 'LIB_ETAT_ARTICLE', 'STATUS']
Shape : (12567, 15)


In [15]:
daily_data

,variant_id,checkout_completed_at,total_demand,stockout_occurred,row_num,demand_7day_avg,stockout_risk_target,DEP,GRP,RAY,store_id,SAISONNALITE_ARTICLE,state,LIB_ETAT_ARTICLE,STATUS
0,392892,2025-07-13,4.0,0,1,NaN,0.0,PRODUIT FRAIS,FOOD,CREMERIE,25,PERMANENT,fulfilled,Actif,found
1,392892,2025-07-17,4.0,0,2,4.0,0.0,PRODUIT FRAIS,FOOD,CREMERIE,25,PERMANENT,fulfilled,Actif,found
2,392893,2025-07-14,1.0,1,1,NaN,NaN,BAZAR,NON FOOD,AUTO-BRICO,25,PERMANENT,fulfilled,Actif,unfound
3,392895,2025-07-11,0.0,0,1,NaN,NaN,PGC,FOOD,EPICERIE,25,PERMANENT,fulfilled,Actif,new/replaced
4,392900,2025-07-12,12.0,0,1,NaN,0.0,PGC,FOOD,ENTRETIEN,25,PERMANENT,fulfilled,Actif,found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5137,3699419,2025-07-13,1.0,0,1,NaN,NaN,NaN,NaN,NaN,25,NaN,fulfilled,NaN,found
5138,3699436,2025-07-17,1.0,0,1,NaN,NaN,NaN,NaN,NaN,25,NaN,fulfilled,NaN,found
5139,3699645,2025-07-16,1.0,0,1,NaN,NaN,NaN,NaN,NaN,25,NaN,fulfilled,NaN,found
5140,3700208,2025-07-12,0.0,0,1,NaN,NaN,NaN,NaN,NaN,25,NaN,fulfilled,NaN,new/replaced


In [34]:
# -------------------------------------------------------------
# Variables calendaires
# -------------------------------------------------------------
daily_data["day_of_week"] = daily_data["checkout_completed_at"].dt.dayofweek   # 0 = lundi … 6 = dimanche
daily_data["month"]       = daily_data["checkout_completed_at"].dt.month       # 1 = janvier … 12 = décembre


In [17]:
daily_data.columns

Index(['variant_id', 'checkout_completed_at', 'total_demand',
       'stockout_occurred', 'row_num', 'demand_7day_avg',
       'stockout_risk_target', 'DEP', 'GRP', 'RAY', 'store_id',
       'SAISONNALITE_ARTICLE', 'state', 'LIB_ETAT_ARTICLE', 'STATUS',
       'day_of_week', 'month'],
      dtype='object')

In [35]:
cols_to_drop = ["SAISONNALITE_ARTICLE", "state", "LIB_ETAT_ARTICLE", "STATUS"]
daily_data.drop(columns=cols_to_drop, inplace=True)

print("✅ Colonnes supprimées :", cols_to_drop)
print("Shape :", daily_data.shape)


✅ Colonnes supprimées : ['SAISONNALITE_ARTICLE', 'state', 'LIB_ETAT_ARTICLE', 'STATUS']
Shape : (12567, 13)


In [19]:
daily_data.columns

Index(['variant_id', 'checkout_completed_at', 'total_demand',
       'stockout_occurred', 'row_num', 'demand_7day_avg',
       'stockout_risk_target', 'DEP', 'GRP', 'RAY', 'store_id', 'day_of_week',
       'month'],
      dtype='object')

In [36]:
# -------------------------------------------------------------
# Ordre cible
# -------------------------------------------------------------
desired_order = [
    "variant_id", "checkout_completed_at", "total_demand",
    "stockout_occurred", "row_num", "demand_7day_avg",
    "stockout_risk_target", "day_of_week", "month",
    "GRP", "DEP", "RAY", "store_id"
]

# -------------------------------------------------------------
# 1) S’assurer que toutes les colonnes existent
# -------------------------------------------------------------
missing = [c for c in desired_order if c not in daily_data.columns]
if missing:
    raise KeyError(f"Colonnes manquantes dans daily_data : {missing}")

# -------------------------------------------------------------
# 2) Réordonner (et, optionnellement, garder les colonnes en trop à la fin)
# -------------------------------------------------------------
# • Juste l’ordre exact, en excluant tout le reste :
daily_data = daily_data[desired_order]

# • OU : garder d’éventuelles colonnes supplémentaires après l’ordre souhaité :
# extra_cols = [c for c in daily_data.columns if c not in desired_order]
# daily_data = daily_data[desired_order + extra_cols]

print("✅ Colonnes réorganisées :", daily_data.columns.tolist())


✅ Colonnes réorganisées : ['variant_id', 'checkout_completed_at', 'total_demand', 'stockout_occurred', 'row_num', 'demand_7day_avg', 'stockout_risk_target', 'day_of_week', 'month', 'GRP', 'DEP', 'RAY', 'store_id']


In [37]:
daily_data.to_csv("data_after_feature_eng_v1.csv", index=False)
print("✅ Fichier enregistré : data_after_feature_eng_v1.csv")


✅ Fichier enregistré : store25_clean.csv
